In [11]:
import requests
from bs4 import BeautifulSoup
import json
import re 

# Accessing the website 
url = "https://www.yelu.in/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Selecting categories
category_links = ["https://www.yelu.in/category/restaurants" , "https://www.yelu.in/category/vehicle-services" ,"https://www.yelu.in/category/schools-india"]

# Scraping company details for each category and Mumbai location
company_list = []
for category_link in category_links:
    location_links = ["city:mumbai"]
    for location_link in location_links:
        for i in range(1, 51):
            url = f"{category_link}/{location_link}?page={i}"
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            companies = soup.find_all("div", class_="company with_img g_0")

            # Extracting company details
            for company in companies:
                if len(company_list) == 1000:
                    break
                company_dict = {}
                company_dict["Name"] = company.find('h4').text.strip()
                company_dict["Address"] = company.find("div", class_="address").text.strip()
                company_dict["Description"] = company.find("div", class_="details").text.strip()
              
                rating_element = company.find("div", class_= "rate")
                if rating_element is not None:
                    rating_text = rating_element.text.strip()
                    rating_float = float(rating_text)
                    company_dict["Rating"] = int(rating_float)
                else:
                    company_dict["Rating"] = "No reviews" # or another default value


                company_dict["Verified"] = "Yes" if "Verified" in company.find("div", class_="cont").text.strip() else "No"
                company_dict["Icon"] = company.find("img")["src"]
                company_dict["Link"] = "https://www.yelu.in" + company.find("a")["href"]
                company_list.append(company_dict)

            if len(company_list) == 1000 or len(company_list) % 500 == 0:
                break

        if len(company_list) == 1000 or len(company_list) % 500 == 0:
            break

    if len(company_list) == 1000 or len(company_list) % 500 == 0:
        break

# Saving data in JSON format
with open("company_data.json", "w") as f:
    json.dump(company_list, f, indent=4)

In [12]:
import pandas as pd
df = pd.DataFrame(company_list)

In [13]:
df

,Name,Address,Description,Rating,Verified,Icon,Link
0,Combo Jumbo,"Shop No. 001, Soho Court, Plot No. 139A, Sindh...",Combo Jumbo is one of the best restaurants in ...,5,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/956077/combo-jumbo
1,"The Oberoi, Mumbai","The Oberoi, Nariman Point, Mumbai - 400 021, M...","The Oberoi, Mumbai enjoys an unrivalled positi...",No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/926961/the-oberoi-...
2,"The Byke Suraj Plaza,Thane","Saibaba Vihar Complex, Anand Nagar, Ghodbunder...","The Byke Suraj Plaza, Luxury Hotel in Thane ha...",No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/937860/the-byke-su...
3,Delhi-Darbar,"Mansuri Caterers SBS Road, Near Regal Cinema, ...",Delhi Darbar is famous restaurants in Mumbai p...,No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/918395/delhi-darbar
4,Queensline SeaYAH,"606, Raheja Chambers, Free Press Journal Road,...",Queensline SeaYAH & Neverland are luxury float...,No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/950312/queensline-...
...,...,...,...,...,...,...,...
995,C P Goenka International School - Borivali,"I.C. Colony, Link Road, Borivali (West), Mumba...","CP Goenka International School, Borivali is an...",No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/954291/c-p-goenka-...
996,JASUDBEN M.L. SCHOOL,"Dr. Madhuri Shah Campus, Ramakrishna Mission R...",J.M.L School also known as Jasudben M.L. Schoo...,No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/945209/jasudben-ml...
997,Ramesh Sippy Academy of Cinema and Entertainment,Garware Institute of Career Education & Develo...,RSACE offers educational courses like animatio...,No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/950441/ramesh-sipp...
998,Swami Vivekanand International School,"Gorai Link Road, Borivali (West), Mumbai, Maha...",Swami Vivekanand International School and Jr. ...,No reviews,Yes,"data:image/gif;base64,R0lGODlhAQABAIABAP///wAA...",https://www.yelu.in/company/941367/swami-vivek...
